In [ ]:
label_to_augment = "ObservedResult"

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    from google.colab import drive

    drive.mount("/content/drive", force_remount=True)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import importlib
import json
import random
import os
import re
from importlib import reload

In [ ]:
if using_colab:
    dir_path = (
        "drive/Othercomputers/my_computer/dl-nlp_project_named-entity-recognition/"
    )
    # dir_path = "drive/MyDrive/dl-nlp_project_named-entity-recognition/"
    module_path = dir_path.replace("/", ".")
    # imports
    data_module = importlib.import_module(module_path + "data")
    load_data = data_module.load_data
    extract_sentences_and_labels = data_module.extract_sentences_and_labels
    generate_label_vocab = data_module.generate_label_vocab
    split_data = data_module.split_data

else:
    dir_path = "./"
    from data import (
        load_data,
        extract_sentences_and_labels,
        generate_label_vocab,
        split_data,
    )

In [ ]:
train_file_path = dir_path + "data/train.json"
test_file_path = dir_path + "data/test.json"

In [ ]:
train_data, test_data = load_data(train_file_path, test_file_path)
train_sentences, train_raw_labels = extract_sentences_and_labels(train_data)
test_sentences, test_raw_labels = extract_sentences_and_labels(test_data)

# Generate label vocabulary
label_vocab = generate_label_vocab(train_raw_labels + test_raw_labels)

In [ ]:
SPECIAL_TOKEN = "<SPC>"


class Labels:
    def __init__(self, num_classes, names):
        super().__init__()
        self.names = names
        print(self.names)
        self.num_classes = num_classes

    def __getitem__(self, label_vector):
        return [self.names[idx] for idx, value in enumerate(label_vector) if value == 1]

    def decode(self, label_vector):
        return self.__getitem__(label_vector)

    def encode(self, names):
        indexes = []
        for name in names:
            index = self.names.index(name)
            indexes.append(index)
        tensor = torch.zeros(self.num_classes)
        for index in indexes:
            tensor[index] = 1
        return tensor

    def tensor2sentence(self, tensor):
        return [self.decode(vector) for vector in tensor]


ner_labels = Labels(
    num_classes=len(label_vocab) + 1, names=label_vocab + [SPECIAL_TOKEN]
)
id2label = ner_labels.decode
label2id = ner_labels.encode
ner_labels.num_classes

In [ ]:
def extract_sentences(json_file_path):
    with open(json_file_path, "r") as file:
        data = json.load(file)

    sentences = []

    for entry in data:
        for sentence in entry["sentences"]:
            tokens = sentence["words"]

            entities = sentence["entities"]
            labels_list = [torch.zeros(ner_labels.num_classes) for x in tokens]
            for label_entity in entities:
                start_pos = label_entity["start_pos"]
                end_pos = label_entity["end_pos"]
                label = label_entity["label"]
                label_id = label2id([label]).argmax().item()
                for label_index in range(start_pos, end_pos + 1):
                    labels_list[label_index][label_id] = 1
            sentence["tokens"] = tokens
            sentence["labels_list"] = labels_list
            sentences.append(sentence)

    return [x["tokens"] for x in sentences], [x["labels_list"] for x in sentences]

In [ ]:
train_sentences, train_labels = extract_sentences(train_file_path)
test_sentences, test_labels = extract_sentences(test_file_path)
train_sentences, train_labels, val_sentences, val_labels = split_data(
    train_sentences, train_labels
)

print(len(train_sentences), len(train_labels))
print(len(val_sentences), len(val_labels))
print(len(test_sentences), len(test_labels))

In [ ]:
data_file_name = f"{dir_path}data/labels/{label_to_augment}.json"
if os.path.exists(data_file_name):
    with open(data_file_name, "r") as json_file:
        data = json.load(json_file)
else:
    data = {
        "sentences": [],
        "labels_lists": [],
    }

In [ ]:
label_abbreviations = {
    "ObjectiveDescription": "OD",
    "Precondition": "PC",
    "RelativeChangeValue": "RCV",
    "DiffGroupAbsValue": "DGAV",
    "NumberPatientsCT": "NPC",
    "AllocationRatio": "AR",
    "DoseValue": "DV",
    "AggregatonMethod": "AM",
    "ResultMeasuredValue": "RMV",
    "SdDevResValue": "SDRV",
    "PvalueDiff": "PDIFF",
    "ConfIntervalChangeValue": "CICV",
    "PValueChangeValue": "PVCV",
    "ConfIntervalDiff": "CID",
    "TimePoint": "TP",
    "PercentageAffected": "PA",
    "NumberAffected": "NA",
    "SubGroupDescription": "SGD",
    "MinAge": "MA",
    "Frequency": "F",
    "ObservedResult": "OR",
    "SdDevChangeValue": "SDCV",
    "FinalNumPatientsArm": "FNPA",
    "DoseDescription": "DD",
    "PublicationYear": "PY",
    "SdDevBL": "SDBL",
    "ConclusionComment": "CC",
    "Journal": "J",
    "AvgAge": "AA",
    "AggregationMethod": "AM",
    "NumberPatientsArm": "NPA",
    "CTDesign": "CTD",
    "Author": "A",
    "Title": "T",
    "Country": "C",
    "Drug": "D",
}
label_unabbreviations = {v: k for k, v in label_abbreviations.items()}

In [127]:
examples = []
examples_with_labels = []
for sentence, labels_list in zip(train_sentences, train_labels):
    new_sentence = sentence
    found = False
    for i, (token, labels) in enumerate(zip(sentence, labels_list)):
        if label_to_augment in id2label(labels):
            found = True
            if not new_sentence[i].startswith("!!"):
                new_sentence[i] = f"!!{token}!!"
    if found:
        # print(" ".join(new_sentence))
        words = []
        for word, labels in zip(sentence, labels_list):
            # print(id2label(labels), word)
            abbreviated_labels = [
                label_abbreviations[label] if label in label_abbreviations else label
                for label in id2label(labels)
            ]
            words.append(f"{word} {abbreviated_labels}")
        # print(words)
        # print()
        examples.append(new_sentence)
        examples_with_labels.append(words)

print(len(examples))

38


## Util Functions


In [ ]:
def tokenize(input_str):
    pattern = r"!![^!]+!!|[-;/\.]|\w+|\S"
    # Find all matches
    tokens = re.findall(pattern, input_str)
    return tokens

In [ ]:
def make_markdown_table(array, align: str = None):
    """
    Args:
        array: The array to make into a table. Mush be a rectangular array
               (constant width and height).
        align: The alignment of the cells : 'left', 'center' or 'right'.
    """
    # make sure every elements are strings
    array = [[str(elt) for elt in line] for line in array]
    # get the width of each column
    widths = [max(len(line[i]) for line in array) for i in range(len(array[0]))]
    # make every width at least 3 colmuns, because the separator needs it
    widths = [max(w, 3) for w in widths]
    # center text according to the widths
    array = [[elt.center(w) for elt, w in zip(line, widths)] for line in array]

    # separate the header and the body
    array_head = array[0]
    array = array[1:]

    header = "| " + " | ".join(array_head) + " |"

    # alignment of the cells
    align = str(align).lower()  # make sure `align` is a lowercase string
    if align == "none":
        # we are just setting the position of the : in the table.
        # here there are none
        border_left = "| "
        border_center = " | "
        border_right = " |"
    elif align == "center":
        border_left = "|:"
        border_center = ":|:"
        border_right = ":|"
    elif align == "left":
        border_left = "|:"
        border_center = " |:"
        border_right = " |"
    elif align == "right":
        border_left = "| "
        border_center = ":| "
        border_right = ":|"
    else:
        raise ValueError("align must be 'left', 'right' or 'center'.")
    separator = (
        border_left + border_center.join(["-" * w for w in widths]) + border_right
    )

    # body of the table
    body = [""] * len(array)  # empty string list that we fill after
    for idx, line in enumerate(array):
        # for each line, change the body at the correct index
        body[idx] = "| " + " | ".join(line) + " |"
    body = "\n".join(body)

    return header + "\n" + separator + "\n" + body

## Code


In [128]:
import ast

index = random.randint(0, len(examples) - 1)
print(" ".join(examples[index]))
# print(examples_with_labels[index])
print()
original_table = []
original_table.append(["Word", "Labels (Abrv.)", "Labels"])

for labels_str in examples_with_labels[index]:
    word = labels_str.split(" ")[0]
    labels = labels_str.split(" ")[1:]
    label_list_str = "".join(labels).lstrip("['").rstrip("']")
    label_list = (
        [label_list_str] if label_list_str.isalpha() else label_list_str.split(",")
    )
    label_list = label_list if label_list != [""] else []
    if len(label_list) > 1:
        for i, label in enumerate(label_list):
            label_list[i] = label.strip("'")
    original_table.append(
        [
            word,
            label_list,
            [
                label_unabbreviations[label]
                if label in label_unabbreviations
                else label
                for label in label_list
            ],
        ]
    )
print(make_markdown_table(original_table, align="left"))

Least - squares mean difference ( 95 % CI ) for HbA1c ( - 0 . 05 [ !!-!! !!0!! !!.!! !!21!! !!,!! !!0!! !!.!! !!11!! ] % ) , glycaemic variability ( 0 . 06 [ !!-!! !!0!! !!.!! !!06!! !!,!! !!0!! !!.!! !!19!! ] mmol / l ) and weight change ( - 0 . 01 [ !!-!! !!0!! !!.!! !!61!! !!,!! !!0!! !!.!! !!59!! ] kg ) showed non - inferiority ( margins of 0 . 4 % , 0 . 8 mmol / l and 1 . 5 kg , respectively ) .

|     Word     | Labels (Abrv.) |         Labels        |
|:------------ |:-------------- |:--------------------- |
|    Least     |       []       |           []          |
|      -       |       []       |           []          |
|   squares    |       []       |           []          |
|     mean     |       []       |           []          |
|  difference  |       []       |           []          |
|      (       |       []       |           []          |
|      95      |       []       |           []          |
|      %       |       []       |           []          |
|      CI      

In [131]:
import time

input_text = input("Input Sentence: ")
token_list = tokenize(input_text.strip('"'))
labels_list = [[] for x in token_list]
for i, token in enumerate(token_list):
    suggesting = i + 1 < len(original_table) and 0 < i + 1
    if suggesting:
        suggestion = (
            f", Suggestion: {original_table[i + 1][1]}, {original_table[i + 1][2]}"
        )
        print(
            f"{i:^3}: {token:^20}{suggestion}",
        )
    else:
        print(f"{i:^3}: {token:^20}")
table = []
table.append(["Id", "Word", "Labels"])
input_label = (
    label_abbreviations[label_to_augment]
    if label_to_augment in label_abbreviations
    else label_to_augment
)
while len(input_label) > 0:
    input_label = input_label.upper()
    start_idx = input(f"Start idx for {input_label}: ")
    while start_idx.isalpha():
        start_idx = input(f"Start idx for {input_label}: ")
    start_idx = int(start_idx)
    end_idx = input(f"End idx for {input_label}: ")
    while end_idx.isalpha():
        end_idx = input(f"End idx for {input_label}: ")
    end_idx = int(end_idx)
    for i in range(start_idx, end_idx + 1):
        labels_list[i].append(
            label_unabbreviations[input_label]
            if input_label in label_unabbreviations
            else input_label
        )
    input_label = input("Label: ")
    while input_label.isnumeric():
        input_label = input("Label: ")

token_list = [token.replace("!!", "") for token in token_list]
print(labels_list, end="\n\n")
for i, (token, labels) in enumerate(zip(token_list, labels_list)):
    labels = list(set(labels))
    table.append([i, token, labels])
print(make_markdown_table(table, align="left"))
print()
time.sleep(1)
submit = input("Hit enter to submit")
if len(submit) == 0:
    data["sentences"].append(token_list)
    data["labels_lists"].append(labels_list)
    print("Data submitted")
else:
    print("Data not submitted")

 0 :         The         , Suggestion: [], []
 1 :      calculated     , Suggestion: [], []
 2 :        least        , Suggestion: [], []
 3 :          -          , Suggestion: [], []
 4 :       squares       , Suggestion: [], []
 5 :         mean        , Suggestion: [], []
 6 :     differences     , Suggestion: [], []
 7 :          (          , Suggestion: [], []
 8 :          95         , Suggestion: [], []
 9 :          %          , Suggestion: [], []
10 :          CI         , Suggestion: [], []
11 :          )          , Suggestion: [], []
12 :         for         , Suggestion: [], []
13 :        HbA1c        , Suggestion: ['DGAV'], ['DiffGroupAbsValue']
14 :          (          , Suggestion: ['DGAV'], ['DiffGroupAbsValue']
15 :          -          , Suggestion: ['DGAV'], ['DiffGroupAbsValue']
16 :          0          , Suggestion: ['DGAV'], ['DiffGroupAbsValue']
17 :          .          , Suggestion: [], []
18 :          05         , Suggestion: ['CID'], ['ConfIntervalDiff']
19 

In [132]:
data_file_name = f"{dir_path}data/labels/{label_to_augment}.json"
with open(data_file_name, "w") as json_file:
    json.dump(data, json_file, indent=2)
    print(f"{data_file_name} was updated.")

./data/labels/ConfIntervalDiff.json was updated.
